# 1)-Importing key Modules



In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install dynet

     |████████████████████████████████| 28.1MB 53.2MB/s 


In [0]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [4]:
! pip install version_information

  Created wheel for version-information: filename=version_information-1.0.3-cp36-none-any.whl size=3880 sha256=36edc0e3ef4e21017c307f1558886f3fe0f02315ef320bd2e60428c0f683cd47
  Stored in directory: /root/.cache/pip/wheels/1f/4c/b3/1976ac11dbd802723b564de1acaa453a72c36c95827e576321
Successfully built version-information


In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,dynet,numpy

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
dynet 2.0.3
numpy 1.17.3
Thu Nov 07 22:01:01 2019 UTC

# 2)- Loading Data

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [0]:
# Read in the data
train = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("test.txt"))

In [8]:
print(len(w2i))
print(len(dev))

18648
2210


In [0]:
nwords = len(w2i)
ntags = len(t2i)

In [10]:
train[0][1]

0

In [0]:
# Start DyNet and define trainer
model = dy.ParameterCollection()
trainer = dy.AdamTrainer(model)

# 3)- Define Model

Using embedding+layer

In [0]:
EMB_SIZE = 64
HID_SIZE = 64
HID_LAY = 2
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_h = [model.add_parameters((HID_SIZE, EMB_SIZE if lay == 0 else HID_SIZE)) for lay in range(HID_LAY)]
b_h = [model.add_parameters((HID_SIZE)) for lay in range(HID_LAY)]
W_sm = model.add_parameters((ntags, HID_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [0]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  h = dy.esum([dy.lookup(W_emb, x) for x in words])
  for W_h_i, b_h_i in zip(W_h, b_h):
    h = dy.tanh( W_h_i * h + b_h_i )
  return W_sm * h + b_sm

# 4)- Train the Model

In [14]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform training
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5136, time=0.61s
iter 0: test acc=0.4032
iter 1: train loss/sent=1.1745, time=0.58s
iter 1: test acc=0.4109
iter 2: train loss/sent=0.8996, time=0.58s
iter 2: test acc=0.4181
iter 3: train loss/sent=0.6380, time=0.58s
iter 3: test acc=0.3919
iter 4: train loss/sent=0.4474, time=0.58s
iter 4: test acc=0.3959
iter 5: train loss/sent=0.3231, time=0.60s
iter 5: test acc=0.3887
iter 6: train loss/sent=0.2327, time=0.58s
iter 6: test acc=0.3991
iter 7: train loss/sent=0.1795, time=0.59s
iter 7: test acc=0.3842
iter 8: train loss/sent=0.1452, time=0.57s
iter 8: test acc=0.3869
iter 9: train loss/sent=0.1109, time=0.58s
iter 9: test acc=0.3910
iter 10: train loss/sent=0.0890, time=0.58s
iter 10: test acc=0.3710
iter 11: train loss/sent=0.0842, time=0.61s
iter 11: test acc=0.3846
iter 12: train loss/sent=0.0737, time=0.59s
iter 12: test acc=0.3706
iter 13: train loss/sent=0.0607, time=0.59s
iter 13: test acc=0.3787
iter 14: train loss/sent=0.0649, time=0.58s
iter 14: t